In [ ]:
import pandas as pd
import nflfastpy as nfl
import seaborn as sns
import matplotlib.pyplot as plt

import requests
from io import BytesIO

In [ ]:
df = nfl.load_pbp_data(2021)

df.head()

In [ ]:
df.groupby('posteam')[['epa']].mean().sort_values(by = 'epa', ascending = False)

In [ ]:
qbs = df.groupby(['passer_player_id', 'passer_player_name'], 
                as_index=False).agg({'play_id': 'count', 'epa': 'mean', 'cpoe': 'mean'})

qbs = qbs.loc[qbs.play_id > 199]

qbs.sort_values('epa', ascending=False, inplace=True)

qbs = qbs.round(2)

qbs.columns = ['ID', 'Player', 'Dropbacks', 'EPA', 'CPOE']

qbs

In [ ]:
df_passes = df.loc[(df['pass_attempt']==1) & (df['air_yards'].notnull()) & (df['down'].isin([3,4]))]

df_passes = df_passes[['passer_player_id', 'passer_player_name', 'posteam', 'down', 'ydstogo', 'air_yards']]

df_passes['yards_past_sticks'] = df_passes['air_yards'] - df_passes['ydstogo']

df_passes.head(10)

In [ ]:
top_10 = df_passes.groupby(['passer_player_id', 'passer_player_name', 'posteam'], 
            as_index=False)['yards_past_sticks'].sum().sort_values(by='yards_past_sticks', ascending=False)[:10]

df_passes = df_passes.loc[df['passer_player_id'].isin(top_10['passer_player_id'])]

df_passes.head(10)

In [ ]:
team_logos_colors = nfl.load_team_logo_data()

df_passes = df_passes.merge(team_logos_colors[['team_abbr', 'team_color', 'team_color2']].
            rename(columns = {'team_abbr': 'posteam'}), on='posteam')

df_passes

In [ ]:
fig, ax = plt.subplots(5,2,figsize = (10,15))

flatten = lambda t: [item for sublist in t for item in sublist]

axes = flatten(ax)

players = [group[-1] for group in df_passes.groupby('passer_player_id')]

for ax, player_df in zip(axes, players):
    sns.kdeplot(player_df['yards_past_sticks'], ax = ax, lw = 4, color = player_df['team_color'].values[0])

    xy = ax.get_lines()[0].get_xydata()
    x, y = xy[:, 0], xy[:, 1]
    ax.fill_between(x,y,color = player_df['team_color'].values[0], alpha = 0.4)

    ax.set_yticks([0, 0.05, 0.1])

    passer_player_name = player_df['passer_player_name'].values[0]
    ax.set_title(f'\n{passer_player_name}', fontsize = 16, fontweight = 450)

    ax.set_ylim(bottom = 0, top = 0.1)
    ax.set_xlim(left = -15, right = 40)

    fig.suptitle("Top 10 QBs in Yards Past Sticks in 2021", fontsize = 18)

    fig.tight_layout()

    fig.subplots_adjust(top = 0.92)

    fig.set_facecolor('white')